In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.multivariate.manova import MANOVA
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import rankdata, chi2

### Analyses of cortical layer thickness between mammalian clades and across developmental stages

This notebook aims to address the statistical considerations regarding the analysis of histological data. 
The histological data compares cortical histological samples taken from two species, *Mus musculus* and *Sminthopsis crassicaudata* and also examines how layer thickness, as a proportion of total cortical thickness, varies across equivalent developmental stages. 


#### Framing the statistical approach

We are interested in comparing the proportional thickness of cortical layers between species and across developmental stages. To this end, we are interested in two factors: species (between-subejcts; 2 levels) and developmental stages (between-subjects; 6 levels). We are working with small-ish numbers and underlying statistical assumptions are the first things to be considered.

##### *What is the precendent?*

While not to most important consideration, precedent established by prior and related published work should be considered. This provides some basis for what is done already in the field and gives us some insight into the collective approach to analysing related data. To begin, the work described by [Hutsler et al. (2005)](https://reader.elsevier.com/reader/sd/pii/S0006899305008528?token=155C8303515DA46D905B5DE6972AD21709B82B04ABF8A9DE8AD8A2F477E520E33E857E7A470303ACCA185281E86789DE&originRegion=us-east-1&originCreation=20230502030258) is a good place to start. They compared rodent and primate sensory cortices, looking both at absolute and proportions of layer thickness. Absolute measurements are informative but are inappropriate for inter-species comparisons. Hence, the authors compare both absolute and proportional measurements. Importantly, for their first comparison (Fig. 2.), they examine only layers II/III and V/VI and they do this using two separate tests.


In [2]:
data = pd.read_excel("/Users/uqdkilpa/Desktop/LauraF/sample_data_dab.xlsx", sheet_name=1)
data

,Condition,WMVZ,L6,L5,L24,MZ
0,CAGCRE,0.000000,74.285714,21.428571,4.285714,0.042857
1,CAGCRE,0.000000,28.571429,46.428571,25.000000,0.250000
2,CAGCRE,14.285714,56.043956,23.076923,6.593407,0.065934
3,CAGCRE,9.090909,53.030303,30.303030,7.575758,7.575758
4,EMX2CRE,0.000000,20.000000,60.000000,20.000000,0.200000
5,EMX2CRE,0.000000,16.666667,33.333333,50.000000,0.500000
6,EMX2CRE,0.000000,9.523810,52.380952,38.095238,0.380952
7,EMX2CRE,0.000000,17.647059,52.941176,29.411765,0.294118


In [22]:
# Fit the MANOVA model using the from_formula method
manova_model = MANOVA.from_formula('VZ + IZ + L56 + L23 ~ Species', data=data)
manova_results = manova_model.mv_test()
print(manova_results.summary())

                                 Multivariate linear model
                                                                                            
--------------------------------------------------------------------------------------------
       Intercept                Value          Num DF  Den DF         F Value         Pr > F
--------------------------------------------------------------------------------------------
          Wilks' lambda                -0.0000 4.0000 11.0000 -12384898975268866.0000 1.0000
         Pillai's trace                 1.0000 4.0000 11.0000 -12384898975268866.0000 1.0000
 Hotelling-Lawley trace -4503599627370497.0000 4.0000 11.0000 -12384898975268866.0000 1.0000
    Roy's greatest root -4503599627370497.0000 4.0000 11.0000 -12384898975268866.0000 1.0000
--------------------------------------------------------------------------------------------
                                                                                            
-----------

In [35]:
# Fit the MANOVA model using the from_formula method
manova_model = MANOVA.from_formula('WMVZ + L6 + L5 + L24 + MZ ~ Condition', data=data)
manova_results = manova_model.mv_test()
print(manova_results.summary())

                                Multivariate linear model
                                                                                         
-----------------------------------------------------------------------------------------
       Intercept                Value          Num DF Den DF        F Value        Pr > F
-----------------------------------------------------------------------------------------
          Wilks' lambda                -0.0000 5.0000 2.0000 -450359962737050.0000 1.0000
         Pillai's trace                 1.0000 5.0000 2.0000 -450359962737050.0000 1.0000
 Hotelling-Lawley trace -1125899906842625.0000 5.0000 1.0000 -225179981368525.0000 1.0000
    Roy's greatest root -1125899906842625.0000 5.0000 2.0000 -450359962737050.0000 1.0000
-----------------------------------------------------------------------------------------
                                                                                         
------------------------------------------

In [ ]:
# SRH test 
# Load your dataset
data = pd.read_csv('your_dataset.csv')

# Replace 'Factor1', 'Factor2', and 'DV' with the appropriate column names in your dataset
# Compute the ranks for the dependent variable (DV)
data['Ranks'] = rankdata(data['DV'])

# Calculate the sum of ranks for each combination of Factor1 and Factor2 levels
rank_sums = data.groupby(['Factor1', 'Factor2'])['Ranks'].sum()

# Calculate the total number of observations (N), levels of Factor1 (A), and levels of Factor2 (B)
N = len(data)
A = len(data['Factor1'].unique())
B = len(data['Factor2'].unique())

# Compute the test statistics H1, H2, and H12
H1 = 12 / (N * (N + 1)) * np.sum(rank_sums.groupby('Factor1').sum() ** 2) / B - 3 * (N + 1)
H2 = 12 / (N * (N + 1)) * np.sum(rank_sums.groupby('Factor2').sum() ** 2) / A - 3 * (N + 1)
H12 = 12 / (N * (N + 1)) * np.sum(rank_sums ** 2) - 3 * (N + 1)

# Compute the error term H3
H3 = H12 - H1 - H2

# Calculate the degrees of freedom for each factor and interaction
df1 = A - 1
df2 = B - 1
df12 = (A - 1) * (B - 1)
df3 = N - A * B

# Compute the chi-square test statistics for each factor and interaction
chi_square_H1 = H1 / df1
chi_square_H2 = H2 / df2
chi_square_H12 = H12 / df12
chi_square_H3 = H3 / df3

# Compute the p-values for each factor and interaction
p_value_H1 = 1 - chi2.cdf(chi_square_H1, df1)
p_value_H2 = 1 - chi2.cdf(chi_square_H2, df2)
p_value_H12 = 1 - chi2.cdf(chi_square_H12, df12)

# Print the results
print("Factor1: Chi-square =", chi_square_H1, "p-value =", p_value_H1)
print("Factor2: Chi-square =", chi_square_H2, "p-value =", p_value_H2)
print("Interaction: Chi-square =", chi_square_H12, "p-value =", p_value_H12)


In [ ]:
# Peform two-way ANOVA with interaction and get the residuals
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load your dataset
data = pd.DataFrame({
    'DV': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'IV1': ['A', 'A', 'B', 'B', 'A', 'A', 'B', 'B', 'A', 'A'],
    'IV2': ['X', 'Y', 'X', 'Y', 'X', 'Y', 'X', 'Y', 'X', 'Y']
})

# Fit the two-way ANOVA model with interaction
model = ols('DV ~ C(IV1) * C(IV2)', data=data).fit()
residuals = model.resid